In [350]:
import dataframes as RAW
import cleaning as CLEAN
import pandas as pd
import numpy as np
import ast

In [351]:
RAW.character_data.head()

,Wiki ID,Freebase ID,Release date,Character name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at release,Map ID,Character ID,Actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [352]:
RAW.movie_data.head()

,Wiki ID,Freebase ID,Movie name,Release date,Revenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988-01-01,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987-01-01,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983-01-01,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [353]:
RAW.summaries.head()

,Wiki ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [354]:
RAW.character_names.head()

,Character name,Freebase ID
0,Stuart Little,/m/0k3w9c
1,Stuart Little,/m/0k3wcx
2,Stuart Little,/m/0k3wbn
3,John Doe,/m/0jyg35
4,John Doe,/m/0k2_zn


In [355]:
RAW.tvtropes.head()

,Character type,Character name,Movie,Freebase ID,Actor name
0,absent_minded_professor,Professor Philip Brainard,Flubber,/m/0jy9q0,Robin Williams
1,absent_minded_professor,Professor Keenbean,Richie Rich,/m/02vchl3,Michael McShane
2,absent_minded_professor,Dr. Reinhardt Lane,The Shadow,/m/0k6fkc,Ian McKellen
3,absent_minded_professor,Dr. Harold Medford,Them!,/m/0k6_br,Edmund Gwenn
4,absent_minded_professor,Daniel Jackson,Stargate,/m/0k3rhh,James Spader


In [356]:
#Make copy 
cleaned_movie_data = RAW.movie_data.copy()

In [357]:
#Filtering-out genres which are NaN
cleaned_movie_data = cleaned_movie_data.dropna(subset=['Genres'])

In [358]:
#Only keeps the values of the dictionaries
def extract_genres(genre_str):
    try:
        genre_dict = ast.literal_eval(genre_str)
        return list(genre_dict.values())
    except (ValueError, SyntaxError) as e:
        print(f"Error converting string to dictionary. Error: {e}")
        return []

In [359]:
#Apply the cleaning function to data. Generates the new "Genres" column
Cleaned_genres = cleaned_movie_data['Genres'].apply(extract_genres)

In [360]:
#Inject the new "Genres" in
#Movie data without NaN and with cleaned up Genres
cleaned_movie_data["Genres"] = Cleaned_genres

In [361]:
#Makes sure to only keep movies with actors in our character metadata
#common_wiki_ids = cleaned_movie_data[cleaned_movie_data["Wiki ID"].isin(RAW.character_data["Wiki ID"])]
result_cleaned_movie_data = cleaned_movie_data[cleaned_movie_data["Wiki ID"].isin(RAW.character_data["Wiki ID"])]

In [376]:
#Counts the numbers of appearance of each genre in all movies
exploded_genres = result_cleaned_movie_data['Genres'].explode()
genre_counts = exploded_genres.value_counts()

In [363]:
# Set your threshold value
threshold = 2000  

# Filter out genres below the threshold
filtered_genres = genre_counts[genre_counts >= threshold].index.tolist()

# Creates filter for the original DataFrame based on the threshold
filtered_genres = result_cleaned_movie_data.apply(lambda row: [genre for genre in row['Genres'] if genre in filtered_genres], axis=1)

In [364]:
#Inject the modified "Genres" in
result_cleaned_movie_data["Genres"] = filtered_genres

<ipython-input-364-6f5c8f9207df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_cleaned_movie_data["Genres"] = filtered_genres


In [365]:
#Counts the counts for each genre in all movies after threshold filter
exploded_genres = result_cleaned_movie_data['Genres'].explode()
genre_counts = exploded_genres.value_counts()

In [377]:
#To visualise the kept genres and their number of appearances
genre_counts

Drama               28490
Comedy              14516
Romance Film         9087
Thriller             8070
Action               7806
Black-and-white      7358
World cinema         7013
Indie                6216
Crime Fiction        5967
Horror               4588
Action/Adventure     4546
Short Film           4510
Adventure            4258
Silent film          3998
Family Film          3644
Musical              3491
Romantic drama       3314
Mystery              2799
Science Fiction      2672
Romantic comedy      2584
Fantasy              2406
Comedy film          2405
Animation            2153
Crime Thriller       2093
War film             2093
Name: Genres, dtype: int64

In [367]:
#List of genres after threshold filtering
unique_genres_list = genre_counts.index.tolist()
unique_genres_list

['Drama',
 'Comedy',
 'Romance Film',
 'Thriller',
 'Action',
 'Black-and-white',
 'World cinema',
 'Indie',
 'Crime Fiction',
 'Horror',
 'Action/Adventure',
 'Short Film',
 'Adventure',
 'Silent film',
 'Family Film',
 'Musical',
 'Romantic drama',
 'Mystery',
 'Science Fiction',
 'Romantic comedy',
 'Fantasy',
 'Comedy film',
 'Animation',
 'Crime Thriller',
 'War film']

In [368]:
#Merging the modified Genres to the characters
character_data_ft_Genres = pd.merge(RAW.character_data, result_cleaned_movie_data[['Wiki ID', 'Genres']], on='Wiki ID', how='left')

In [370]:
#Deletes any row which does not have any genre after threshold filtering
character_data_ft_Genres = character_data_ft_Genres[character_data_ft_Genres['Genres'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

In [372]:
#Creates the Genre columns filled with value 0
unique_genres_list

for Genre in unique_genres_list:
    character_data_ft_Genres[Genre] = 0

In [373]:
#Adds 1 if the Genre of the character's movie is present
for Genre in unique_genres_list:
    character_data_ft_Genres[Genre] = character_data_ft_Genres['Genres'].apply(lambda x: 1 if Genre in x else 0)

In [374]:
# Initialize a dictionary to store genre counts for each actor
actor_genre_counts = {}

# Iterate over each row and update the genre counts for each actor
for index, row in character_data_ft_Genres.iterrows():
    actor_name = row['Actor name']
    genres = row['Genres']
    
    if actor_name not in actor_genre_counts:
        actor_genre_counts[actor_name] = {genre: 0 for genre in unique_genres_list}
    
    for genre in genres:
        if genre in unique_genres_list:
            actor_genre_counts[actor_name][genre] += 1

# Create a new DataFrame from the dictionary
actor_genre_counts_df = pd.DataFrame.from_dict(actor_genre_counts, orient='index')

In [375]:
#Final dataframe
actor_genre_counts_df

,Drama,Comedy,Romance Film,Thriller,Action,Black-and-white,World cinema,Indie,Crime Fiction,Horror,...,Musical,Romantic drama,Mystery,Science Fiction,Romantic comedy,Fantasy,Comedy film,Animation,Crime Thriller,War film
Wanda De Jesus,6,1,0,6,5,0,0,2,6,1,...,0,0,1,2,0,0,0,0,5,0
Natasha Henstridge,10,5,5,8,8,0,1,0,8,4,...,0,2,0,5,2,1,0,0,3,0
Ice Cube,13,13,1,6,10,0,0,1,8,2,...,0,0,0,1,0,0,1,0,3,1
Jason Statham,13,5,2,23,23,0,1,1,17,1,...,1,1,1,3,0,2,0,1,11,0
Clea DuVall,18,9,6,11,2,0,0,9,6,5,...,0,1,7,4,4,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Violet Bronte,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
Sonny Byrkett,2,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,2,0,0
Susan Byrkett,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
Hal Cleaveland,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
